<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/114_1_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一）
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
Sheet 欄位：date, category, item, amount, payer

google sheet：https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing

In [47]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [48]:
import pandas as pd
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

In [49]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的資料
df.head()

,日期,時間,項目,金額


In [50]:
from google.colab import auth
import gspread
from google.auth import default
from datetime import datetime

# 步驟 1: 進行 Google 驗證並連接到試算表
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

# 步驟 2: 取得特定的工作表 (Worksheet)
worksheet = gsheets.sheet1  # 或使用 gsheets.worksheet('工作表名稱')

print("--- 輸入資料 ---")

# 步驟 3: 獲取使用者輸入並進行驗證
while True:
    date_str = input("請輸入日期 (格式：YYYY-MM-DD)：")
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        break
    except ValueError:
        print("日期格式不正確，請重新輸入。")

while True:
    time_str = input("請輸入時間 (格式：HH:MM)：")
    try:
        datetime.strptime(time_str, '%H:%M')
        break
    except ValueError:
        print("時間格式不正確，請重新輸入。")

while True:
    item_str = input("請輸入項目：")
    if item_str.strip() == "":
        print("項目不能為空，請重新輸入。")
    else:
        break

while True:
    amount_str = input("請輸入金額 (必須是數字)：")
    try:
        amount_float = float(amount_str)
        break
    except ValueError:
        print("金額格式不正確，請重新輸入數字。")

# 步驟 4: 將驗證後的資料組合成一行清單
new_row = [date_str, time_str, item_str, amount_float]

print("\n--- 資料驗證成功 ---")
print(f"輸入的資料為：{new_row}")

# 步驟 5: 附加新資料到 Google 試算表
try:
    worksheet.append_rows([new_row])
    print("\n✅ 資料已成功寫入 Google 試算表！")
except Exception as e:
    print(f"\n寫入資料時發生錯誤：{e}")

--- 輸入資料 ---
請輸入日期 (格式：YYYY-MM-DD)：2000-02-02
請輸入時間 (格式：HH:MM)：02:02
請輸入項目：djd
請輸入金額 (必須是數字)：3000

--- 資料驗證成功 ---
輸入的資料為：['2000-02-02', '02:02', 'djd', 3000.0]

✅ 資料已成功寫入 Google 試算表！
